In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/ind_nifty50list.csv')
pd.DataFrame({'label':df['Symbol'],'value':df['Symbol']}).to_dict(orient='records')

In [20]:
pd.DataFrame({'label':df['Symbol'],'value':df['Symbol']}).to_dict(orient='records')

[{'label': 'ADANIPORTS', 'value': 'ADANIPORTS'},
 {'label': 'ASIANPAINT', 'value': 'ASIANPAINT'},
 {'label': 'AXISBANK', 'value': 'AXISBANK'},
 {'label': 'BAJAJ-AUTO', 'value': 'BAJAJ-AUTO'},
 {'label': 'BAJFINANCE', 'value': 'BAJFINANCE'},
 {'label': 'BAJAJFINSV', 'value': 'BAJAJFINSV'},
 {'label': 'BPCL', 'value': 'BPCL'},
 {'label': 'BHARTIARTL', 'value': 'BHARTIARTL'},
 {'label': 'INFRATEL', 'value': 'INFRATEL'},
 {'label': 'BRITANNIA', 'value': 'BRITANNIA'},
 {'label': 'CIPLA', 'value': 'CIPLA'},
 {'label': 'COALINDIA', 'value': 'COALINDIA'},
 {'label': 'DRREDDY', 'value': 'DRREDDY'},
 {'label': 'EICHERMOT', 'value': 'EICHERMOT'},
 {'label': 'GAIL', 'value': 'GAIL'},
 {'label': 'GRASIM', 'value': 'GRASIM'},
 {'label': 'HCLTECH', 'value': 'HCLTECH'},
 {'label': 'HDFCBANK', 'value': 'HDFCBANK'},
 {'label': 'HEROMOTOCO', 'value': 'HEROMOTOCO'},
 {'label': 'HINDALCO', 'value': 'HINDALCO'},
 {'label': 'HINDUNILVR', 'value': 'HINDUNILVR'},
 {'label': 'HDFC', 'value': 'HDFC'},
 {'label':